In [92]:
repo_path = !git rev-parse --show-toplevel
assert (len(repo_path) == 1)

exec_path = repo_path[0] + "/_build/default/test/benchmark_process_packet.exe"

In [162]:
import numpy as np
import json

def run(scheduler,domains=-1,spawners=-1):
    cmd = exec_path + " -scheduler " + scheduler + " -num-of-domains " + str(domains) + " -num-of-spawners " + str(spawners)
    output = !$cmd
    assert (output[0][0:5] == "start")
    assert (output[-1][0:4] == "done")
    output = output[1:-1]
    
    grouped_time = [] 
    grouped_latency = []
    grouped_executed_tasks = []
    
    for i in range(1,round(len(output)/4)):
        index = i*4
        
        
        #assert (output[index][:10] == "iteration:")
        #iteration = int(output[index][10:])
        assert (output[index+1][:5] == "time:")
        time = int(output[index+1][5:]) 
        grouped_time.append(time)
        
        assert (output[index+2][:8] == "latency:")
        latency = output[index+2][8:]
        grouped_latency.append(json.loads(latency))
        
        assert (output[index+3][:15] == "executed-tasks:")
        executed_tasks = output[index+3][15:]
        grouped_executed_tasks.append(json.loads(executed_tasks))
        
        
        #grouped.append([iteration, time, np.array(json.loads(latency)),np.array(json.loads(exec_tasks))])
        
    avg_time = np.mean(grouped_time)
    avg_latency = np.mean(grouped_latency,axis=0)
    
    overall_execution_mean = np.mean(grouped_executed_tasks,axis=1)[0]
    execution_dist_err = grouped_executed_tasks - overall_execution_mean
    execution_err = np.mean(np.sqrt(np.mean(np.square(execution_dist_err),axis=1)))
    
    return (avg_time, avg_latency, execution_err)

In [163]:
out = run("FIFO", domains=3, spawners=1)

In [164]:
(grouped_time, grouped_latency, grouped_executed_tasks) = out

In [165]:
out

(424.0,
 array([0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 6.60000e+00, 4.65000e+01,
        9.62000e+01, 2.94100e+02, 1.93380e+03, 4.43970e+03, 8.73550e+03,
        1.79165e+04, 3.62518e+04, 2.62595e+04, 2.85390e+03, 9.78300e+02,
        1.88300e+02, 3.00000e-01, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00]),
 5713.636127400552)

In [166]:
import matplotlib.pyplot as plt